In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ranked-board-game-data-from-boardgamegeek/mechanisms_2023.csv
/kaggle/input/ranked-board-game-data-from-boardgamegeek/reimplementations_2023.csv
/kaggle/input/ranked-board-game-data-from-boardgamegeek/themes_2023.csv
/kaggle/input/ranked-board-game-data-from-boardgamegeek/subdomains_2023.csv
/kaggle/input/ranked-board-game-data-from-boardgamegeek/basic_data_2023.csv


In [3]:
# emojis
!pip install emoji --upgrade
import emoji

  Obtaining dependency information for emoji from https://files.pythonhosted.org/packages/8d/97/fbe537350214b0489e6c7052b9e8928a85ed5febb621a82cc5437dbf17e7/emoji-2.10.0-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.9/457.9 kB 6.1 MB/s eta 0:00:0000:010:01
  Attempting uninstall: emoji
    Found existing installation: emoji 2.9.0
    Uninstalling emoji-2.9.0:
      Successfully uninstalled emoji-2.9.0


In [4]:
# similar names
from fuzzywuzzy import process


In [5]:
# randomize akinator questions
from random import randint

# Reading datasets

In [6]:
df_games = pd.read_csv("/kaggle/input/ranked-board-game-data-from-boardgamegeek/basic_data_2023.csv", 
                       index_col='game_id')
upper_limit_id_games = df_games.index.max()
min_limit_id_games = df_games.index.min()

In [7]:
df_mechs = pd.read_csv("/kaggle/input/ranked-board-game-data-from-boardgamegeek/mechanisms_2023.csv",
                      index_col = 'game_id')

In [8]:
df_themes = pd.read_csv('/kaggle/input/ranked-board-game-data-from-boardgamegeek/themes_2023.csv',
                       index_col = 'game_id')


In [9]:
df_subdomains = pd.read_csv("/kaggle/input/ranked-board-game-data-from-boardgamegeek/subdomains_2023.csv",
                           index_col = 'game_id')


# Choose an function option

In [10]:
#initial menu
option = 4 # when finish, change to None
wanted_id = None
while True:
    break #clear after finish
    try:
        option = int(input("""
        Choose an option to search a game:
            [1] from ID;
            [2] from similar names
            [3] randomly
            [4] from mechanics or themes
            [5] Boardgame Akinator
            [0] None, just exit the program\n"""))
    except ValueError:
        print(f'Please, type a valid option.... \n{"-"*30}')
    
    if option in [0,1,2,3,4,5]:
        break
    else:
        print(f'Please, type an number on menu.')

# [1] Searching from ID

In [11]:
#if def functions, erase this cell
continuing = 'Y'

while continuing == 'Y' and option == 1:
    
    #Structure to ensure game_id does exist
    wanted_id = upper_limit_id_games+1
    while df_games['name'].get(wanted_id) == None:
        try:
            wanted_id = int(input(f"Digit a game ID between {min_limit_id_games} and {upper_limit_id_games}): "))
        except ValueError:
            print(f'Please, type a valid number.... \n{"-"*30}')
        while (wanted_id > upper_limit_id_games or wanted_id < min_limit_id_games):
            wanted_id = int(input(f"Digit a game ID between {min_limit_id_games} and {upper_limit_id_games}): "))
        wanted_name = df_games['name'].get(wanted_id)
        if wanted_name == None:
            print("This code doesn't match to any game...")
            wanted_sup = wanted_id
            wanted_inf = wanted_id
            while df_games['name'].get(wanted_sup) == None:
                wanted_sup = wanted_sup + 1

            while df_games['name'].get(wanted_inf) == None:
                wanted_inf = wanted_inf - 1


            print(f"The nearer numbers are {wanted_inf} or {wanted_sup}") 
        else:

            break

    print(f"The game with the ID {wanted_id} is the {wanted_name}.")

   
    continuing = str.upper(input("Would you like to select again? [Y/N]"))
    if continuing == 'N':
        break

# [2] Searching from similar names

In [12]:
similar_game = 0
while similar_game == 0 and option == 2:
    game = input("Which game are you looking for?")
    all_games = df_games['name']

#compare names
    results = process.extract(game,all_games, limit=10)
    print(f"the names similars to {game} are: ")
    for similar_game in range(len(results)):
        print(f' [{similar_game+1}] - {results[similar_game][0]}')
    similar_game = int(input('Choose an option: "0" to type again'))
    wanted_id = int((df_games.loc[df_games.name == results[similar_game-1][0]].index)[0])


# [3] Random Game

In [13]:
# call a random valid name
while option == 3:
    wanted_id = randint(min_limit_id_games, upper_limit_id_games+1)
    if df_games.name.get(wanted_id) != None: # não erro
        break
print (wanted_id, df_games.name.get(wanted_id))     

None None


# [4] From Mechanics or Themes

In [31]:
# create lists mechs and themes
list_mechs = df_mechs.columns.tolist()
temp_mechs_list = ["mechanic: " + mech for mech in list_mechs]
list_themes = df_themes.columns.tolist()
temp_themes_list = ["theme: " + theme for theme in list_themes]
list_characteristics = temp_mechs_list + temp_themes_list


In [67]:
similar_characteristic = 0

while similar_characteristic == 0 and option == 4:
    #read characteristic
    characteristic = input("Which characteristic are you looking for?")
    
    #compare caracteristic
    results = process.extract(characteristic,list_characteristics,limit=10)
    
    #show results
    print(f"The characteristics similar to {characteristic} are: ")
    for similar_characteristic in range(len(results)):
        print(f' [{similar_characteristic+1}] - {results[similar_characteristic][0]}')
    
    # choose characteristic
    try:
        similar_characteristic = int(input('Choose an option or type "0" to try again: '))
    except ValueError:
        print(f'Please, type a valid number.... \n{"-"*30}')
        
      # PASSOU COM ERRO  ^  
        
        
    wanted_chararacteristic = results[similar_characteristic-1][0].split(":")[1].strip()
# remove 'mechanics:' or 'themes:' and strip to remove space
#show games with this charac
    try:
        wanted_id = df_mechs.index[df_mechs[wanted_chararacteristic] ==1]
    except KeyError:
        wanted_id = df_themes.index[df_themes[wanted_chararacteristic] ==1]

#picking a game
print('\n','*-*'*20)
print(f'These are the game list with {wanted_chararacteristic}:')
for game in wanted_id:
    print(f'Game ID ({game}) - {df_games.name[game]}')
while True:
    while True:
        try:
            wanted_id = int(input('Pick a game from its index: '))
        except ValueError:
            print('WARNING! Choose a game ID.')
        finally:
            break
    if df_games['name'].get(wanted_id) != None:
        break

Which characteristic are you looking for? tower


The characteristics similar to tower are: 
 [1] - mechanic: Cube Tower
 [2] - mechanic: Ownership
 [3] - mechanic: Variable Player Powers
 [4] - mechanic: Action Timer
 [5] - mechanic: Advantage Token
 [6] - mechanic: Auction: Turn Order Until Pass
 [7] - mechanic: Bids As Wagers
 [8] - mechanic: Cooperative Game
 [9] - mechanic: Grid Coverage
 [10] - mechanic: Hidden Victory Points


Choose an option or type "0" to try again:  d


Please, type a valid number.... 
------------------------------

 *-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-*
These are the game list with Grid Coverage:
Game ID (84876) - The Castles of Burgundy
Game ID (177736) - A Feast for Odin
Game ID (163412) - Patchwork
Game ID (281259) - The Isle of Cats
Game ID (263918) - Cartographers
Game ID (199561) - Sagrada
Game ID (283155) - Calico
Game ID (295486) - My City
Game ID (555) - The Princes of Florence
Game ID (286096) - Tapestry
Game ID (227789) - Heaven & Ale
Game ID (13122) - Antiquity
Game ID (184921) - Bunny Kingdom
Game ID (219513) - Bärenpark
Game ID (265736) - Tiny Towns
Game ID (315767) - Cartographers Heroes
Game ID (265402) - In the Hall of the Mountain King
Game ID (270673) - Silver & Gold
Game ID (318009) - Dinosaur Island: Rawr 'n Write
Game ID (126792) - Myrmes
Game ID (283863) - The Magnificent
Game ID (300877) - New York Zoo
Game ID (2453) - Blokus
Game ID (276498) - Paris: La Cité de la Lumière
Game ID (3185

Pick a game from its index:  g


WARNING! Choose a game ID.


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

# [5] Teste Akinator

In [ ]:
# comparison formule
def total_players(valor):
    #merging tables
    print(df_games.shape)
    df_intermediate = df_games.merge(df_themes, how = 'left', on = df_games.index)
    df_total = df_intermediate.merge(df_mechs, how = 'left')
    df_total.shape
    # Substitua esta função pela sua lógica de comparação
    if df_total.min_players <= players and df_total.max_players >= players:
            df_total['akinator'] = df_total['akinator'].apply(lambda x: x + 10)

    return valor > 10

# Aplica a fórmula à coluna 'akinator'


In [ ]:
df_total['akinator'] = 0
# condition to continue the questions
while df_total['akinator'].nunique() != 1 or df_total['akinator'].max() != df_total['akinator'].iloc[0] and option == 5:
    # Realize as perguntas aqui, atualizando o DataFrame df_total conforme necessário
    players = input("How many players are you looking for?")
        
    # Pode ser uma entrada do usuário ou outro método de atualização
    df_total['comparacao'] = df_total['akinator'].apply(total_players)

    # Atualize os valores na coluna 'akinator' para simular uma mudança nos valores

# Imprima o DataFrame quando a condição for atendida
#print("DataFrame final:")
#print(df_total)

# Printing results

In [65]:
# MECHS/THEMES/SUBDOMAINS: Select the row from the index
wanted_mechs = df_mechs.loc[wanted_id]
wanted_themes = df_themes.loc[wanted_id]
wanted_subdomains = df_subdomains.loc[wanted_id]

# MECHS/THEMES/SUBDOMAINS: Obtain the nemas of columns whete from the index
wanted_mechs = wanted_mechs.index[wanted_mechs == 1].tolist()
wanted_themes = wanted_themes.index[wanted_themes == 1].tolist()
wanted_subdomains = wanted_subdomains.index[wanted_subdomains == 1].tolist()

# MECHS
df_similar_mechs = df_mechs[wanted_mechs]
df_similar_mechs['sum'] = df_similar_mechs.sum(axis=1)

# MECHS: list of 2 top sums
top2_sums_mechs = df_similar_mechs['sum'].unique().tolist()
top2_sums_mechs.sort(reverse=True)
top2_sums_mechs = top2_sums_mechs[0:2]

# MECHS: exclude 0'es in list 
if 0 in top2_sums_mechs:
    top2_sums_mechs.remove(0)

# MECHS: games with 2 top sums    
only_most_similar_games_mechs = df_similar_mechs.loc[df_similar_mechs['sum'].isin(top2_sums_mechs)].index.tolist()

# MECHS: Remove elements equal to search item
only_most_similar_games_mechs.remove(wanted_id)
only_most_similar_games_mechs = df_games.name.loc[only_most_similar_games_mechs].tolist()

# THEMES
df_similar_themes = df_themes[wanted_themes]
df_similar_themes['sum'] = df_similar_themes.sum(axis=1)


# THEMES: list of 2 top sums
top2_sums_themes = df_similar_themes['sum'].unique().tolist()
top2_sums_themes.sort(reverse=True)
top2_sums_themes = top2_sums_themes[0:2]

# THEMES: exclude 0'es in list 
if 0 in top2_sums_themes:
    top2_sums_themes.remove(0)

# THEMES: games with 2 top sums    
only_most_similar_games_themes = df_similar_themes.loc[df_similar_themes['sum'].isin(top2_sums_themes)].index.tolist()

#  THEMES: Remove os elementos iguais ao valor especificado
only_most_similar_games_themes.remove(wanted_id)
only_most_similar_games_themes = df_games.name.loc[only_most_similar_games_themes].tolist()


/tmp/ipykernel_42/2849690178.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_similar_mechs['sum'] = df_similar_mechs.sum(axis=1)
/tmp/ipykernel_42/2849690178.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_similar_themes['sum'] = df_similar_themes.sum(axis=1)


In [66]:
input("Press ENTER to see the results")
print(15*'-*-')
print(emoji.emojize(f":bullseye: The game with the code {wanted_id} is {df_games['name'].get(wanted_id)}, link {df_games.bgg_url[wanted_id]}. \n"))
print(15*'-*-')

print(emoji.emojize("📗"))
print(df_games.loc[wanted_id,['rank','min_players', 'max_players', 'avg_time','min_time', 
'max_time', 'weight', 'year', 'age', 'avg_rating','geek_rating', 'num_votes', 
'owned', 'designer']])

print(15*'-*-')

print(f"The subdomains are: {wanted_subdomains}")
print(15*'-*-')

print(emoji.emojize(f"🛠 MECHANICS \nThe mechanics in this game are:"))
for item in range(len(wanted_mechs)):
    print(f"  - {wanted_mechs[item]}")
    
print(f'The games with similar mechanics are: ')
for item in range(len(only_most_similar_games_mechs)):
    print(f'  - {only_most_similar_games_mechs[item]} - year ', end = "")
    print(df_games.loc[df_games.name == only_most_similar_games_mechs[item],'year'].iloc[0], end = "")
    print(" - weight (0 to 5) ", end = "")
    print(df_games.loc[df_games['name'] == only_most_similar_games_mechs[item],'weight'].iloc[0])
print(15*'-*-')

print(emoji.emojize(f"\n 📜 THEMES \nThe themes in this game are:"))
for item in range(len(wanted_themes)):
    print(f"  - {wanted_themes[item]}") 
print(f'The games with similar themes are: ')
for item in range(len(only_most_similar_games_themes)):
    print(f'  - {only_most_similar_games_themes[item]} - ano ', end = "")
    print(df_games.loc[df_games.name == only_most_similar_games_themes[item],'year'].iloc[0], end = "")
    print(" - peso ", end = "")
    print(df_games.loc[df_games['name'] == only_most_similar_games_themes[item],'weight'].iloc[0])



Press ENTER to see the results 


-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
🎯 The game with the code 316412 is The LOOP, link https://boardgamegeek.com/boardgame/316412. 

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
📗
rank                                     615
min_players                                1
max_players                                4
avg_time                                  60
min_time                                  60
max_time                                  60
weight                                  2.77
year                                    2020
age                                       12
avg_rating                           7.78773
geek_rating                          6.84396
num_votes                               3090
owned                                   7001
designer       Maxime Rambourg, Théo Rivière
Name: 316412, dtype: object
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
The subdomains are: ['Strategy Games']
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
🛠 MECHANICS 
The mech